### Protein Family Classification

In [1]:
import numpy as np
import pandas as pd

import sklearn.utils
from sklearn.linear_model import LogisticRegression

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf

from lazy import lazy

import collections
import re

In [2]:
family_classification_metadata = pd.read_table('../seminar_5/data/family_classification_metadata.tab')
family_classification_sequences = pd.read_table('../seminar_5/data/family_classification_sequences.tab')

In [3]:
family_classification_metadata.head()

,SwissProtAccessionID,LongID,ProteinName,FamilyID,FamilyDescription
0,Q6GZX4,001R_FRG3G,Putative transcription factor 001R,Pox_VLTF3,Poxvirus Late Transcription Factor VLTF3 like
1,Q6GZX3,002L_FRG3G,Uncharacterized protein 002L,DUF230,Poxvirus proteins of unknown function
2,Q6GZX0,005R_FRG3G,Uncharacterized protein 005R,US22,US22 like
3,Q91G88,006L_IIV6,Putative KilA-N domain-containing protein 006L,DUF3627,Protein of unknown function (DUF3627)
4,Q197F3,007R_IIV3,Uncharacterized protein 007R,DUF2738,Protein of unknown function (DUF2738)


In [4]:
family_classification_sequences.head()

,Sequences
0,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...
1,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...
2,MQNPLPEVMSPEHDKRTTTPMSKEANKFIRELDKKPGDLAVVSDFV...
3,MDSLNEVCYEQIKGTFYKGLFGDFPLIVDKKTGCFNATKLCVLGGK...
4,MEAKNITIDNTTYNFFKFYNINQPLTNLKYLNSERLCFSNAVMGKI...


In [5]:
family_classification_metadata.describe()

,SwissProtAccessionID,LongID,ProteinName,FamilyID,FamilyDescription
count,324018,324018,324018,324018,324018
unique,287308,295671,56951,7027,6967
top,Q1X881,POLG_DEN3M,UvrABC system protein B,MMR_HSR1,50S ribosome-binding GTPase
freq,16,12,1500,3084,3084


#### Task:
    
Use your ProtVec embedding from homework 5 to perform protein family classification using RNN.

Article with the original research can be found here http://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0141287&type=printable

* use 1000 most frequent families for classification
* validate your results on the train-test split
* reduce the dimensionality of the protein-space using Stochastic Neighbor Embedding and visualize two most frequent classes
* compare your RNN results with SVM
* visualization and metrics are up to you

In [6]:
# Used to limit performance when cluster is loaded
CLUSTER_LIMIT = 10 ** 9

In [7]:
family_ids = np.array(family_classification_metadata['FamilyID'])
# Finding top-k by frequency: https://stackoverflow.com/a/19909411/5338270
unique_ids, positions = np.unique(family_ids, return_inverse=True)
top_thousand_indices = np.bincount(positions).argsort()[::-1][:1000]
top_two_indices = np.bincount(positions).argsort()[::-1][:2]
top_two_mask = np.in1d(positions, top_two_indices)[:CLUSTER_LIMIT]
top_thousand_mask = np.in1d(positions, top_thousand_indices)
top_thousand_families = unique_ids[top_thousand_indices]

In [8]:
sequences_to_classify = np.array(family_classification_sequences)[top_thousand_mask][:CLUSTER_LIMIT]
family_ids_gt = family_ids[top_thousand_mask][:CLUSTER_LIMIT]
family_ids_gt = sklearn.preprocessing.LabelEncoder().fit_transform(family_ids_gt).reshape((len(sequences_to_classify), 1))
family_ids_gt = sklearn.preprocessing.OneHotEncoder(sparse=False).fit_transform(family_ids_gt)
print('Number of sequences: {}'.format(len(sequences_to_classify)))
print('Max length of sequence: {}'.format(np.apply_along_axis(lambda x: len(x[0]), 1, sequences_to_classify).max()))

Number of sequences: 261149
Max length of sequence: 22152


In [9]:
BATCH_SIZE = 64
NUM_CLASSES = 1000
MAX_RAW_LEN = 500 #np.apply_along_axis(lambda x: len(x[0]), 1, sequences_to_classify).max()
MAX_SEQ_LEN = (MAX_RAW_LEN + 2) // 3
NUM_EPOCHS = 1
EMBED_LEN = 100

In [10]:
# Pad sequences with spaces
sequences_to_classify = np.apply_along_axis(lambda x: x[0].ljust(MAX_RAW_LEN), 1, sequences_to_classify)

In [11]:
class MyEmbedding:
    def __init__(self):
        # Dirty code
        self._dict = dict()
        with open('final_embedding.txt', 'r') as fl:
            inp = fl.read().strip()
            
            while inp:
                trigram = inp[:3]
                start_arr = inp.index('[')
                end_arr = inp.index(']')
                values = eval(re.sub('\s+', ',', inp[start_arr+1:end_arr-1].strip()))
                values = np.array(values)
                self._dict[trigram] = values
                inp = inp[end_arr + 1:].lstrip()
    
    def lookup(self, trigram):
        return self._dict.get(trigram, np.zeros((100,)))

In [ ]:
embedding = MyEmbedding()

In [ ]:
# Transform strings to real sequences of vectors
def get_words(s):
    return [embedding.lookup(s[i:i+3]) for i in range(0, min(len(s), MAX_RAW_LEN), 3)]

sequences_to_classify = np.array(list(map(lambda s: get_words(s), sequences_to_classify)))

In [ ]:
tsne = TSNE()
printable_seq = sequences_to_classify[top_two_mask]
printable_seq = np.mean(printable_seq, axis=2)

XX = tsne.fit_transform(printable_seq)

plt.figure()
plt.scatter(XX[:, 0], XX[:, 1], c=['red' if x == 'Helicase_C' else 'green' for x in family_ids[top_two_mask]])
plt.show()

In [ ]:
class DNASequenceClassifier:
    def __init__(self, rnn_hidden_dim, learning_rate, gradient_clipping):
        self._rnn_hidden_dim = rnn_hidden_dim
        self._learning_rate = learning_rate
        self._gradient_clipping = gradient_clipping
        
        self._create_placeholders()
        
        self.prediction
        self.cost
        self.error
        self.optimize
        
        self._create_summaries()
        
    def _create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar('loss', self.cost)
            tf.summary.scalar('error', self.error)
            self.summary = tf.summary.merge_all()
            self.saver = tf.train.Saver()
        
    def _create_placeholders(self):
        with tf.name_scope('data'):
            self.data = tf.placeholder(tf.float32, [None, MAX_SEQ_LEN, EMBED_LEN])
            self.target = tf.placeholder(tf.float32, [None, NUM_CLASSES])
        
    @staticmethod
    def _last_relevant(output):
        with tf.name_scope("last_relevant"):
            batch_size = tf.shape(output)[0]
            output_size = int(output.get_shape()[2])
            index = tf.range(0, batch_size) * MAX_SEQ_LEN + (MAX_SEQ_LEN - 1)
            flat = tf.reshape(output, [-1, output_size])
            relevant = tf.gather(flat, index)
        return relevant        
        
    @lazy
    def length(self):
        with tf.name_scope("seq_length"):
            used = tf.sign(tf.reduce_max(tf.abs(self.data), reduction_indices=2))
            length = tf.reduce_sum(used, reduction_indices=1)
            length = tf.cast(length, tf.int32)
        return length        
        
    @lazy
    def prediction(self):
        with tf.name_scope('rnn'):
            output, _ = tf.nn.dynamic_rnn(
                tf.contrib.rnn.GRUCell(self._rnn_hidden_dim),
                self.data,
                dtype=tf.float32,
                sequence_length=self.length
            )
            
            # [batch_size, time, dims]
            #last_out = tf.gather(output, MAX_SEQ_LEN - 1, axis=1)
            last_out = self._last_relevant(output)
            
            with tf.name_scope("softmax_layer"):
                weight = tf.Variable(tf.truncated_normal(
                    [self._rnn_hidden_dim, NUM_CLASSES], stddev=0.01))
                bias = tf.Variable(tf.constant(0.1))
                self._logits = tf.matmul(last_out, weight) + bias
                prediction = tf.nn.softmax(self._logits)
                
                return prediction
        
    @lazy
    def cost(self):
        with tf.name_scope('softmax'):
            return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.target,
                                                      logits=self._logits))
        
    @lazy
    def error(self):
        with tf.name_scope('accuracy'):
            self.mistakes = tf.not_equal(
                tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
            return tf.reduce_mean(tf.cast(self.mistakes, tf.float32))
        
    @lazy
    def optimize(self):
        with tf.name_scope("optimization"):
            optimizer=tf.train.AdamOptimizer(self._learning_rate)
            gradient = optimizer.compute_gradients(self.cost)
            limit = self._gradient_clipping
            gradient = [
                (tf.clip_by_value(g, -limit, limit), v)
                if g is not None else (None, v)
                for g, v in gradient]
            optimize = optimizer.apply_gradients(gradient)
            
        return optimize

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sequences_to_classify, family_ids_gt)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
baseline_cls = LogisticRegression()
baseline_cls.fit(np.mean(X_train, axis=2), np.argmax(y_train, axis=1))
print('Baseline score on test set: {}%'.format(100 * baseline_cls.score(np.mean(X_test, axis=2), 
                                                                 np.argmax(y_test, axis=1))))

In [ ]:
model = DNASequenceClassifier(rnn_hidden_dim=150, learning_rate=1.9, gradient_clipping=100)

In [ ]:
config = tf.ConfigProto(
    device_count = {'GPU': 0},
)

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    ckpt = tf.train.get_checkpoint_state('checkpoints')
    if ckpt:
        model.saver.restore(sess, ckpt.model_checkpoint_path)

    for epoch in range(NUM_EPOCHS):
        sklearn.utils.shuffle(X_train, y_train)
        for i in range(0, len(X_train), BATCH_SIZE):
            feed_dict = {model.data: X_train[i:i+BATCH_SIZE], model.target: y_train[i:i+BATCH_SIZE]}
            cost, error, _, summary_str = sess.run([model.cost, model.error, model.optimize, model.summary], feed_dict=feed_dict)
            print('{} {}: {:3.1f}% loss={}'.format(epoch, i, 100 * (1 - error), cost))
        model.saver.save(sess, 'checkpoints/rnn', epoch)

    
    feed_dict = {model.data: X_test, model.target: y_test}
    
    total_accuracy, total_loss = sess.run([model.error, model.cost], feed_dict)
    print('Statistics on test set\nAccuracy: {}, loss: {}'.format(1 - total_accuracy, total_loss))